In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import stats
from matplotlib import rcParams
import itertools
import math
from pathlib import Path
from typing import Any, Dict, List

# Define the font 
plt.rcParams['font.family'] = 'Computer Modern'  # Replace 'serif' with the name of the desired font
# Define colorblind-friendly palette
palette = sns.color_palette("colorblind")




In [ ]:
def plot_single_csv_with_loss(folder_path, csv_file):
    '''
    Plots the validation loss for a specific CSV file.

        Parameters:
                folder_path (string): Path to the folder containing the CSV files
                csv_file (string): Name of the CSV file
    '''
    # Read the CSV file
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    
    # Extract the epochs and validation loss as numpy arrays
    epochs = data['Epoch'].values
    validation_loss = data['Validation_Loss'].values
    seed = data["SEED"][0]
    learning_rate = data["AE_LEARNING_RATE"][0]
    batch_size = data["AE_BATCH_SIZE"][0]
    test_loss = data["Test_Loss"][0]
    
    # Plot the graph
    plt.plot(epochs, validation_loss)
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    plt.title(f'Validation Loss - {seed} - {learning_rate} - {batch_size}')
    plt.suptitle(f'Test Loss: {test_loss}')
    plt.grid(True)
    plt.show()

In [ ]:
def plot_single_csv_with_accuracy(folder_path, csv_file):
    '''
    Plots the validation acc for a specific CSV file.

        Parameters:
                folder_path (string): Path to the folder containing the CSV files
                csv_file (string): Name of the CSV file
    '''
    # Read the CSV file
    file_path = os.path.join(folder_path, csv_file)
    data = pd.read_csv(file_path)
    
    # Extract the epochs and validation loss as numpy arrays
    epochs = data['Epoch'].values
    validation_loss = data['Validation_Loss'].values
    validation_accuracy = data['Validation_Accuracy'].values
    seed = data["SEED"][0]
    learning_rate = data["AE_LEARNING_RATE"][0]
    batch_size = data["AE_BATCH_SIZE"][0]
    test_acc = data["Test_Accuracy"][0]
    
    # Plot the graph
    plt.plot(epochs, validation_accuracy)
    plt.xlabel('Epoch')
    plt.ylabel('Validation Acc')
    plt.title(f'Validation Acc - {seed} - {learning_rate} - {batch_size}')
    plt.suptitle(f'Test Acc: {test_acc}')
    plt.grid(True)
    plt.show()

In [ ]:
def plot_validation_accuracy(folder_path, model):
    '''
    Plots the average validation accuracy for a specific model.
    
        Parameters:
                folder_path (string): Path to the folder containing the CSV files
                model (string): Name of the model
    '''

    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Filter out the CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    val_accuracy_list = []
    val_loss_list = []


    for file in csv_files:
        # Read the CSV file
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        # check if file is the model we are looking for
        model_list_name = data["List Name"][0]
        if model_list_name == model:
            model_learning_rate = data["LEARNING_RATE"][0]            
            model_test_accuracy = data["Test_Accuracy"][0]
            model_dataset_name = data["Dataset"][0]
            model_seed = data["SEED"][0]
            model_validation_accuracy = data["Validation_Accuracy"].values
            model_validation_loss = data["Validation_Loss"].values
            
            # model_validation_accuracy = model_validation_accuracy[:50]
            # model_validation_loss = model_validation_loss[:50]

            val_accuracy_list.append((model_seed, model_validation_accuracy))
            val_loss_list.append((model_seed, model_validation_loss))

    val_accuracy_list = sorted(val_accuracy_list, key=lambda x:x[0])
    val_loss_list = sorted(val_loss_list, key=lambda x:x[0])

    
    # Create a figure and axis
    fig, ax = plt.subplots()

    # Average over all seeds
    # Extract the array from the tuple
    array_data_accuracy = [arr for _, arr in val_accuracy_list]
    # Calculate the average for each corresponding element in the arrays
    acc_averages = np.mean(array_data_accuracy, axis=0)
    ax.plot(acc_averages, label=f"Average Accuracy")

    array_data_loss = [arr for _, arr in val_loss_list]
    # Calculate the average for each corresponding element in the arrays
    loss_averages = np.mean(array_data_loss, axis=0)
    # ax.plot(loss_averages, label=f"Average Loss")

    # Add labels and title
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss and Accuracy')
    ax.set_title(f'{model_dataset_name} - {model}')
    # Add legend
    ax.legend()

    # Display the graph
    plt.show()

In [ ]:
plt.rcParams["text.usetex"] = True
rc("font", **{"family": "serif", "serif": ["Computer Modern"]})
rc("text", usetex=True)
sns.set_style("whitegrid")
sns.set(font_scale=1.5)

sns.set(style="whitegrid", font="Computer Modern", font_scale=2)


# Define the color palette for the models
model_color_palette = sns.color_palette("colorblind")
relevant_model_names = ["dressed_classical_list", "dressed_quantum_list", "sequent_classical_list", "sequent_quantum_list"]

# Create a custom color dictionary to map each model name to its corresponding color
model_colors = {
    'dressed_quantum_list': (0.00784313725490196, 0.6196078431372549, 0.45098039215686275), 
    'sequent_quantum_list': (0.8352941176470589, 0.3686274509803922, 0.0), 
    'dressed_classical_list': (0.984313725490196, 0.6862745098039216, 0.8941176470588236), 
    'sequent_classical_list': (0.5803921568627451, 0.5803921568627451, 0.5803921568627451)
    }

def plot_validation_accuracy_and_loss_all_models_Transfer_Learning(folder_path, dataset_name):
    '''
    Plots the validation accuracy and validation loss with error bands for specific models.

    Parameters:
        folder_path (string): Path to the folder containing the CSV files
        dataset_name (string): Name of the dataset for which to show the legend
    '''

    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Filter out the CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    val_acc_data = []
    val_loss_data = []

    for file in csv_files:
        # Read the CSV file
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        # Check if the file is for a relevant model
        model_list_name = data["List Name"][0]

        if model_list_name in relevant_model_names:
            model_seed = data["SEED"][0]
            model_validation_accuracy = data["Validation_Accuracy"].values
            model_validation_loss = data["Validation_Loss"].values

            for epoch, (accuracy, loss) in enumerate(zip(model_validation_accuracy, model_validation_loss)):
                val_acc_data.append({
                    'Seed': model_seed,
                    'Validation Accuracy': accuracy,
                    'Validation Loss': loss,
                    'Epoch': epoch,
                    'Model': model_list_name
                })    
    
    # Create DataFrame from the list
    df = pd.DataFrame(val_acc_data)

    # Combine classical and quantum models and add 100 to the epoch for quantum models
    val_acc_data_combined = []
    for data in val_acc_data:
        if "classical" in data["Model"]:
            val_acc_data_combined.append(data)
        elif "quantum" in data["Model"]:
            data_quantum = data.copy()
            if dataset_name == "MNIST" or dataset_name == "Audio MNIST" :
                data_quantum["Epoch"] += 49
            else:
                data_quantum["Epoch"] += 99
            val_acc_data_combined.append(data_quantum)

    # Create DataFrame from the combined data
    df_combined = pd.DataFrame(val_acc_data_combined)

    # Create a line plot for validation accuracy
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    '''
    sns.lineplot(data=df_combined, x='Epoch', y='Validation Accuracy', hue='Model', err_style='band', linewidth=1.5,
                 palette=model_colors, hue_order=relevant_model_names)
    '''


    alpha =0.6
    df_combined['Smoothed Accuracy'] = df_combined.groupby('Model')['Validation Accuracy'].transform(lambda x: x.ewm(alpha = alpha).mean())
    df_combined['Smoothed Loss'] = df_combined.groupby('Model')['Validation Loss'].transform(lambda x: x.ewm(alpha =alpha).mean())


    # Create a line plot with error bands representing the standard deviation for validation accuracy
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    '''
    sns.lineplot(data=df, x='Epoch', y='Validation Accuracy', hue='Model', errorbar='sd', linewidth=1.5,
                 palette=model_colors, hue_order=relevant_model_names)
    '''
    sns.lineplot(x='Epoch', y='Smoothed Accuracy', hue='Model', data=df_combined, errorbar='sd', linewidth=1.5, palette=model_colors, hue_order=relevant_model_names)


    # print(df_combined)
    # Add labels and title for validation accuracy plot
    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    # plt.title(f'Validation Accuracy for {dataset_name}')

    # Show the legend only for the banknote dataset
    """
    if dataset_name == "Banknote":
        mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
        handles, previous_labels = plt.gca().get_legend_handles_labels()
        plt.legend(handles=handles, labels=mylabels, fontsize=10, loc='lower right')
    elif dataset_name == "Audio MNIST":
        mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
        handles, previous_labels = plt.gca().get_legend_handles_labels()
        plt.legend(handles=handles, labels=mylabels, fontsize=10, loc='lower right')
    """
    mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
    handles, previous_labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles=handles, labels=mylabels, fontsize=10, bbox_to_anchor=(1.02, 1), loc='upper left')
        # plt.legend('',frameon=False)
    '''   
    if dataset_name == "Banknote":
        mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
        handles, previous_labels = plt.gca().get_legend_handles_labels()
        plt.legend(handles=handles, labels=mylabels, fontsize=10, loc='lower right')
    elif dataset_name == "Audio MNIST":
        mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
        handles, previous_labels = plt.gca().get_legend_handles_labels()
        plt.legend(handles=handles, labels=mylabels, fontsize=10, loc='lower right')
    '''


    # Set x-axis limits to start at 0 and end at the maximum epoch
    max_epoch = df_combined['Epoch'].max()
    plt.xlim(left=0)
    plt.xlim(right=max_epoch+2)


    # Display the validation accuracy graph
    plt.show()

    # Create a line plot for validation loss
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    '''
    sns.lineplot(data=df_combined, x='Epoch', y='Validation Loss', hue='Model', errorbar='sd', linewidth=1.5,
                 palette=model_colors, hue_order=relevant_model_names)
    '''
    sns.lineplot(x='Epoch', y='Smoothed Loss', hue='Model', data=df_combined, errorbar='sd', linewidth=1.5, palette=model_colors, hue_order=relevant_model_names)

    # Add labels and title for validation loss plot
    plt.xlabel('Epoch')
    plt.ylabel('Validation Loss')
    # plt.title(f'Validation Loss for {dataset_name}')

    # Show the legend for validation loss plot
    mylabels = ["DQC (classical)", "DQC (quantum)", "SEQUENT (classical)", "SEQUENT (quantum)"]
    handles, previous_labels = plt.gca().get_legend_handles_labels()
    plt.legend(handles=handles, labels=mylabels, fontsize=10, bbox_to_anchor=(1.02, 1), loc='upper left')


    # Set x-axis limits to start at 0 and end at the maximum epoch
    plt.xlim(left=0)#, right=max_epoch)
    plt.xlim(right=max_epoch+2)

    # Display the validation loss graph
    plt.show()



# Define the folder paths for each dataset
folder_path_BanknoteAuthentication = os.path.join(os.getcwd(), "..", "Results", "Banknote")
folder_path_BreastCancer = os.path.join(os.getcwd(), "..", "Results", "Breast_Cancer")
folder_path_MNIST = os.path.join(os.getcwd(), "..", "Results", "MNIST")
folder_path_Audio = os.path.join(os.getcwd(), "..", "Results", "Audio_MNIST")


# Plot validation accuracy and validation loss for each dataset
plot_validation_accuracy_and_loss_all_models_Transfer_Learning(folder_path_BanknoteAuthentication, "Banknote")
plot_validation_accuracy_and_loss_all_models_Transfer_Learning(folder_path_BreastCancer, "Breast Cancer")
plot_validation_accuracy_and_loss_all_models_Transfer_Learning(folder_path_MNIST, "MNIST")
plot_validation_accuracy_and_loss_all_models_Transfer_Learning(folder_path_Audio, "Audio MNIST")


In [ ]:
def plot_validation_accuracy_all_models_with_errorband_wo_custom_legend(folder_path):
    '''
    Plots the validation accuracy with error bands for specific models.

    Parameters:
        folder_path (string): Path to the folder containing the CSV files
    '''

    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Filter out the CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    val_acc_data = []

    for file in csv_files:
        # Read the CSV file
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        # Check if the file is for a relevant model
        model_list_name = data["List Name"][0]

        relevant_model_names = ["VQC_angle_list", "VQC_amplitude_list", "dressed_quantum_list", "sequent_quantum_list", "NN_with_compressed_input_list",  "NN_with_original_input_list"]
        if model_list_name in relevant_model_names:
            model_seed = data["SEED"][0]
            model_validation_accuracy = data["Validation_Accuracy"].values

            for epoch, accuracy in enumerate(model_validation_accuracy):
                val_acc_data.append({
                    'Seed': model_seed,
                    'Validation Accuracy': accuracy,
                    'Epoch': epoch,
                    'Model': model_list_name
                })

    # Create DataFrame from the list
    df = pd.DataFrame(val_acc_data)

    # Set the plot style and font size
    sns.set(font_scale=1.5)
    sns.set_style("whitegrid")

    # Create a line plot with error bands representing the standard deviation
    plt.figure(figsize=(10, 6))  # Adjust the figure size as needed
    sns.lineplot(data=df, x='Epoch', y='Validation Accuracy', hue='Model', errorbar='sd', linewidth=1.5,
                 palette=model_color_palette, hue_order=relevant_model_names)

    # Add labels and title
    plt.xlabel('Epoch')
    plt.ylabel('Validation Accuracy')
    plt.title(f'Model Validation Accuracy for {os.path.basename(folder_path)}')

    # Show the legend
    # mylabels = ["VQC with angle encoding", "VQC with amplitude encoding", "Dressed Quantum NN", "Sequent Quantum NN", "NN with compressed input", "NN with original input"]

    plt.legend(fontsize=10)

    # Display the graph
    plt.show()

# Define the folder paths for each dataset
folder_path_Audio = os.path.join(os.getcwd(), "..", "Results", "Audio_MNIST")
folder_path_BanknoteAuthentication = os.path.join(os.getcwd(), "..", "Results", "Banknote")
folder_path_BreastCancer = os.path.join(os.getcwd(), "..", "Results", "Breast_Cancer")
folder_path_MNIST = os.path.join(os.getcwd(), "..",  "Results", "MNIST")

# Plot validation accuracy for each dataset
plot_validation_accuracy_all_models_with_errorband_wo_custom_legend(folder_path_BanknoteAuthentication)
plot_validation_accuracy_all_models_with_errorband_wo_custom_legend(folder_path_BreastCancer)
plot_validation_accuracy_all_models_with_errorband_wo_custom_legend(folder_path_MNIST)
plot_validation_accuracy_all_models_with_errorband_wo_custom_legend(folder_path_Audio)


In [ ]:
def test_accuracy_confidence_interval(folder_path, model):
    '''
    Calculates the mean test accuracy and the 95% confidence interval for a given model.
        Parameters:
                folder_path (string): Path to the folder containing the CSV files
                model (string): Name of the model
    '''
    
    # Get all the files in the folder
    files = os.listdir(folder_path)
    # Filter out the CSV files
    csv_files = [file for file in files if file.endswith('.csv')]

    test_accuracy_list = []

    for file in csv_files:
        # Read the CSV file
        file_path = os.path.join(folder_path, file)
        data = pd.read_csv(file_path)
        # check if file is the model we are looking for
        model_list_name = data["List Name"][0]
        if model_list_name == model:
            model_seed = data["SEED"][0]
            model_test_accuracy = data["Test_Accuracy"][0]
            test_accuracy_list.append((model_seed, model_test_accuracy))

    test_accuracy_list = sorted(test_accuracy_list, key=lambda x: x[0])
    # Extract the array from the tuple
    array_data_accuracy = np.array([acc for _, acc in test_accuracy_list])

    # Calculate mean test accuracy and standard error (SE) of the mean
    mean_test_accuracy = np.mean(array_data_accuracy)
    se_test_accuracy = stats.sem(array_data_accuracy)

    # Calculate the 95% confidence interval for test accuracy
    confidence_interval = stats.t.interval(0.95, len(array_data_accuracy) - 1, loc=mean_test_accuracy, scale=se_test_accuracy)

    print(abs(mean_test_accuracy-confidence_interval[0]))
    print(abs(mean_test_accuracy-confidence_interval[1]))
    # Print the Results
    print("Model:", model)
    print("Mean Test Accuracy:", mean_test_accuracy)
    print("95% Confidence Interval for Test Accuracy:", confidence_interval)


In [ ]:
def calculate_accuracy_list_for_all_datasets(folder_paths, model_list, output_csv, round_numbers):
    '''
    Calculates the mean test accuracy and the 95% confidence interval for all models in the model list for all datasets.
    The results are stored in the output CSV file. The accuracies for each model and dataset over all seeds are also stored in the CSV file as a list. 
        Parameters:
                folder_paths (dict): Dictionary containing the folder paths for each dataset
                model_list (list): List of models for which to calculate the confidence interval
                output_csv (string): Name of the output CSV file
                round_numbers (bool): Whether to round the numbers to 3 decimal places
    '''
    
    all_Results = []

    for dataset, folder_path in folder_paths.items():
        for model in model_list:
            # Get all the files in the folder
            files = os.listdir(folder_path)
            # Filter out the CSV files
            csv_files = [file for file in files if file.endswith('.csv')]

            test_accuracy_list = []

            for file in csv_files:
                # Read the CSV file
                file_path = os.path.join(folder_path, file)
                data = pd.read_csv(file_path)
                # check if file is the model we are looking for
                model_list_name = data["List Name"][0]
                if model_list_name == model:
                    model_seed = data["SEED"][0]
                    model_test_accuracy = data["Test_Accuracy"][0]
                    test_accuracy_list.append((model_seed, model_test_accuracy))

            # Sort the list by the seed
            test_accuracy_list = sorted(test_accuracy_list, key=lambda x: x[0])
            # Extract the array from the tuple
            array_data_accuracy = np.array([acc for _, acc in test_accuracy_list])


            # Calculate mean test accuracy and standard error (SE) of the mean
            mean_test_accuracy = np.mean(array_data_accuracy)
            se_test_accuracy = stats.sem(array_data_accuracy)

            # Calculate the 95% confidence interval for test accuracy
            confidence_interval_acc = stats.t.interval(0.95, len(array_data_accuracy) - 1, loc=mean_test_accuracy, scale=se_test_accuracy)
            conf_interval_half_acc = abs(confidence_interval_acc[1] - confidence_interval_acc[0]) / 2


            if round_numbers:
                # round to 3 decimal places
                mean_test_accuracy = round(mean_test_accuracy, 3)
                conf_interval_half_acc = round(conf_interval_half_acc, 3)


            all_Results.append({"Model": model,
                                "Dataset": dataset,
                                "Accuracies": array_data_accuracy,
                                "Mean test acc": mean_test_accuracy,
                                "Confidence interval acc": confidence_interval_acc,
                                "Confidence interval / 2 acc": conf_interval_half_acc})
    # Create a DataFrame from the Results for all datasets
    df_all_Results = pd.DataFrame(all_Results)

    # Save the DataFrame to a CSV file
    df_all_Results.to_csv(output_csv, index=False)


In [ ]:
# check if all files are in the folder
folder_path_BanknoteAuthentication= os.path.join(os.getcwd(), "..", "Results", "Banknote")
folder_path_BreastCancer = os.path.join(os.getcwd(), "..", "Results", "Breast_Cancer")
folder_path_MNIST = os.path.join(os.getcwd(), "..", "Results", "MNIST")
folder_path_Audio = os.path.join(os.getcwd(), "..", "Results", "Audio_MNIST")


# Example usage
folder_paths = {
    "BanknoteAuthentication": folder_path_BanknoteAuthentication,
    "BreastCancer": folder_path_BreastCancer,
    "MNIST": folder_path_MNIST,
    "Audio": folder_path_Audio
}
model_list = [
    "VQC_angle_testing_list",
    "VQC_amplitude_testing_list",
    "dressed_quantum_testing_list",
    "sequent_quantum_testing_list",
    "NN_with_compressed_input_testing_list",
    "NN_with_original_input_testing_list"
]

output_csv = os.path.join(os.getcwd(), "..", "Results", "accuracy_list_all_models_datasets.csv")

calculate_accuracy_list_for_all_datasets(folder_paths, model_list, output_csv, round_numbers=False)

In [ ]:
def calculate_accuracy_loss_confidence_interval_for_all_datasets(folder_paths, model_list, output_csv, round_numbers):
    '''
    Calculates the mean test accuracy and test loss and the 95% confidence interval for all models in the model list for all datasets.
    The results are stored in the output CSV file.
    
        Parameters:
            folder_paths (dict): Dictionary containing the folder paths for each dataset
            model_list (list): List of models for which to calculate the confidence interval
            output_csv (string): Name of the output CSV file
            round_numbers (bool): Whether to round the numbers to 3 decimal places
    '''



    all_Results = []

    for dataset, folder_path in folder_paths.items():
        for model in model_list:
            # Get all the files in the folder
            files = os.listdir(folder_path)
            # Filter out the CSV files
            csv_files = [file for file in files if file.endswith('.csv')]

            test_accuracy_list = []
            test_loss_list = []

            for file in csv_files:
                # Read the CSV file
                file_path = os.path.join(folder_path, file)
                data = pd.read_csv(file_path)
                # check if file is the model we are looking for
                model_list_name = data["List Name"][0]
                if model_list_name == model:
                    model_seed = data["SEED"][0]
                    model_test_accuracy = data["Test_Accuracy"][0]
                    test_accuracy_list.append((model_seed, model_test_accuracy))
                    model_test_loss = data["Test_Loss"][0]
                    test_loss_list.append((model_seed, model_test_loss))

            test_accuracy_list = sorted(test_accuracy_list, key=lambda x: x[0])
            # Extract the array from the tuple
            array_data_accuracy = np.array([acc for _, acc in test_accuracy_list])

            # Calculate mean test accuracy and standard error (SE) of the mean
            mean_test_accuracy = np.mean(array_data_accuracy)
            se_test_accuracy = stats.sem(array_data_accuracy)

            # Calculate the 95% confidence interval for test accuracy
            confidence_interval_acc = stats.t.interval(0.95, len(array_data_accuracy) - 1, loc=mean_test_accuracy, scale=se_test_accuracy)
            conf_interval_half_acc = abs(confidence_interval_acc[1] - confidence_interval_acc[0]) / 2


            test_loss_list = sorted(test_loss_list, key=lambda x: x[0])
            # Extract the array from the tuple
            array_data_loss = np.array([loss for _, loss in test_loss_list])

            # Calculate mean test loss and standard error (SE) of the mean
            mean_test_loss = np.mean(array_data_loss)
            se_test_loss = stats.sem(array_data_loss)

            # Calculate the 95% confidence interval for test loss
            confidence_interval_loss = stats.t.interval(0.95, len(array_data_loss) - 1, loc=mean_test_loss, scale=se_test_loss)
            conf_interval_half_loss = abs(confidence_interval_loss[1] - confidence_interval_loss[0]) / 2


            if round_numbers:
                # round to 3 decimal places
                mean_test_accuracy = round(mean_test_accuracy, 3)
                conf_interval_half_acc = round(conf_interval_half_acc, 3)
                mean_test_loss = round(mean_test_loss, 3)
                conf_interval_half_loss = round(conf_interval_half_loss, 3)


            all_Results.append({"Model": model,
                                "Dataset": dataset,
                                "Mean test acc": mean_test_accuracy,
                                "Confidence interval acc": confidence_interval_acc,
                                "Confidence interval / 2 acc": conf_interval_half_acc,
                                "Mean test loss": mean_test_loss,
                                "Confidence interval loss": confidence_interval_loss,
                                "Confidence interval / 2 loss": conf_interval_half_loss})
    # Create a DataFrame from the Results for all datasets
    df_all_Results = pd.DataFrame(all_Results)

    # Save the DataFrame to a CSV file
    df_all_Results.to_csv(output_csv, index=False)


In [ ]:
# Example usage
folder_paths = {
    "BanknoteAuthentication": folder_path_BanknoteAuthentication,
    "BreastCancer": folder_path_BreastCancer,
    "MNIST": folder_path_MNIST,
    "Audio": folder_path_Audio
}
model_list = [
    "VQC_angle_testing_list",
    "VQC_amplitude_testing_list",
    "dressed_quantum_testing_list",
    "sequent_quantum_testing_list",
    "NN_with_compressed_input_testing_list",
    "NN_with_original_input_testing_list"
]

output_csv = os.path.join(os.getcwd(), "..", "Results", "all_models_test_acc_confidence_interval.csv")

calculate_accuracy_loss_confidence_interval_for_all_datasets(folder_paths, model_list, output_csv, round_numbers=False)

In [ ]:
def overall_test_acc(csv_file, round_numbers):
    '''
    Calculates the mean test accuracy and the 95% confidence interval over all models in the model list for all datasets.
    I.e. the average test accuracy over all models for each dataset.
    The results are stored in a new CSV file.

        Parameters:
                csv_file (string): Name of the input CSV file
                round_numbers (bool): Whether to round the numbers to 3 decimal places
    '''

    all_Results = []
    data = pd.read_csv(csv_file)
    models = data["Model"]
    datasets = data["Dataset"]
    mean_test_accuracies = data["Mean test acc"]

    acc_VQC_angle = []
    acc_VQC_amplitude = []
    acc_dressed = []
    acc_sequent = []
    acc_nn_compressed = []
    acc_nn_original = []

    for i in range(len(models)):
        if(models[i] == "VQC_angle_testing_list"):
            acc_VQC_angle.append(mean_test_accuracies[i])
        elif(models[i] == "VQC_amplitude_testing_list"):
            acc_VQC_amplitude.append(mean_test_accuracies[i])
        elif(models[i] == "dressed_quantum_testing_list"):
            acc_dressed.append(mean_test_accuracies[i])
        elif(models[i] == "sequent_quantum_testing_list"):
            acc_sequent.append(mean_test_accuracies[i])
        elif(models[i] == "NN_with_compressed_input_testing_list"):
            acc_nn_compressed.append(mean_test_accuracies[i])
        elif(models[i] == "NN_with_original_input_testing_list"):
            acc_nn_original.append(mean_test_accuracies[i])
    


    # Calculate mean test accuracy and standard error (SE) of the mean
    mean_VQC_angle = np.mean(acc_VQC_angle)
    mean_VQC_amplitude = np.mean(acc_VQC_amplitude)
    mean_dressed = np.mean(acc_dressed)
    mean_sequent = np.mean(acc_sequent)
    mean_nn_compressed = np.mean(acc_nn_compressed)
    mean_nn_original = np.mean(acc_nn_original)

    se_VQC_angle = stats.sem(acc_VQC_angle)
    se_VQC_amplitude = stats.sem(acc_VQC_amplitude)
    se_dressed = stats.sem(acc_dressed)
    se_sequent = stats.sem(acc_sequent)
    se_nn_compressed = stats.sem(acc_nn_compressed)
    se_nn_original = stats.sem(acc_nn_original)


    # Calculate the 95% confidence interval for test accuracy
    CI_VQC_angle = stats.t.interval(0.95, len(acc_VQC_angle) - 1, loc=mean_VQC_angle, scale=se_VQC_angle)
    CI_VQC_amplitude = stats.t.interval(0.95, len(acc_VQC_amplitude) - 1, loc=mean_VQC_amplitude, scale=se_VQC_amplitude)
    CI_dressed = stats.t.interval(0.95, len(acc_dressed) - 1, loc=mean_dressed, scale=se_dressed)
    CI_sequent = stats.t.interval(0.95, len(acc_sequent) - 1, loc=mean_sequent, scale=se_sequent)
    CI_nn_compressed = stats.t.interval(0.95, len(acc_nn_compressed) - 1, loc=mean_nn_compressed, scale=se_nn_compressed)
    CI_nn_original = stats.t.interval(0.95, len(acc_nn_original) - 1, loc=mean_nn_original, scale=se_nn_original)

    # Calculate the confidence interval half
    conf_interval_half_VQC_angle = abs(CI_VQC_angle[1] - CI_VQC_angle[0]) / 2
    conf_interval_half_VQC_amplitude = abs(CI_VQC_amplitude[1] - CI_VQC_amplitude[0]) / 2
    conf_interval_half_dressed = abs(CI_dressed[1] - CI_dressed[0]) / 2
    conf_interval_half_sequent = abs(CI_sequent[1] - CI_sequent[0]) / 2
    conf_interval_half_nn_compressed = abs(CI_nn_compressed[1] - CI_nn_compressed[0]) / 2
    conf_interval_half_nn_original = abs(CI_nn_original[1] - CI_nn_original[0]) / 2


    if round_numbers:
        # round to 3 decimal places
        mean_VQC_angle = round(mean_VQC_angle, 3)
        mean_VQC_amplitude = round(mean_VQC_amplitude, 3)
        mean_dressed = round(mean_dressed, 3)
        mean_sequent = round(mean_sequent, 3)
        mean_nn_compressed = round(mean_nn_compressed, 3)
        mean_nn_original = round(mean_nn_original, 3)

        conf_interval_half_VQC_angle = round(conf_interval_half_VQC_angle, 3)
        conf_interval_half_VQC_amplitude = round(conf_interval_half_VQC_amplitude, 3)
        conf_interval_half_dressed = round(conf_interval_half_dressed, 3)
        conf_interval_half_sequent = round(conf_interval_half_sequent, 3)
        conf_interval_half_nn_compressed = round(conf_interval_half_nn_compressed, 3)
        conf_interval_half_nn_original = round(conf_interval_half_nn_original, 3)


    # store as csv
    all_Results.append({"Model": "VQC_angle",
                    "Mean test acc": mean_VQC_angle,
                    "Confidence interval acc": CI_VQC_angle,
                    "Confidence interval / 2 acc": conf_interval_half_VQC_angle})
    all_Results.append({"Model": "VQC_amplitude",
                    "Mean test acc": mean_VQC_amplitude,
                    "Confidence interval acc": CI_VQC_amplitude,
                    "Confidence interval / 2 acc": conf_interval_half_VQC_amplitude})
    all_Results.append({"Model": "Dressed",
                    "Mean test acc": mean_dressed,
                    "Confidence interval acc": CI_dressed,
                    "Confidence interval / 2 acc": conf_interval_half_dressed})
    all_Results.append({"Model": "Sequent",
                    "Mean test acc": mean_sequent,
                    "Confidence interval acc": CI_sequent,
                    "Confidence interval / 2 acc": conf_interval_half_sequent})
    all_Results.append({"Model": "NN_compressed",
                    "Mean test acc": mean_nn_compressed,
                    "Confidence interval acc": CI_nn_compressed,
                    "Confidence interval / 2 acc": conf_interval_half_nn_compressed})
    all_Results.append({"Model": "NN_original",
                    "Mean test acc": mean_nn_original,
                    "Confidence interval acc": CI_nn_original,
                    "Confidence interval / 2 acc": conf_interval_half_nn_original})

     # Create a DataFrame from the Results for all datasets
    df_all_Results = pd.DataFrame(all_Results)

    # Save the DataFrame to a CSV file
    output = os.path.join(os.getcwd(), "..", "Results", "all_models_test_acc_confidence_interval_summary.csv")
    

    df_all_Results.to_csv(output, index=False)



In [ ]:
overall_test_acc(output_csv, True)